# LISTA 7 - OBTENCIÓN DE DATOS

## <ins>Ejercicios Obligatorios </ins>

In [2]:
!pip install Faker #es una herramienta para generar datos falsos de forma automática dentro de una aplicación
!pip install PyMySQL #permite la interacción con bases de datos MySQL
import codecs
from datetime import date
from datetime import datetime
from faker import Faker
import pymysql
import random
import requests
import csv
import pandas as pd
from sqlalchemy import create_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.5 MB/s eta 0:00:00


## EJERCICIO 1

In [9]:
NUMERO_CLIENTES = 500
NUMERO_PROVEEDORES = 10
SEMILLA_ALEATORIA_GENERADOR = 10
SEMILLA_ALEATORIA_RANDOM = 1

Faker.seed(SEMILLA_ALEATORIA_GENERADOR)
random.seed(SEMILLA_ALEATORIA_RANDOM)
fake = Faker(['es_ES'])

def build_providers_dataset(number):
  providers = []
  for i in range(1, number+1):
    providers.append({
      "provider_id": i,
      "name": fake.company(),
      "email": fake.company_email(),
      "webpage": fake.domain_name()
    })

  return {
      "providers": providers
  }

def build_products_dataset(providers_info):
  products = []
  url = 'https://drive.google.com/uc?export=view&id=1D9MY0au4b7SXwhUdm6TNfsKfYzdkbAh_'
  content = requests.get(url)
  text = codecs.iterdecode(content.iter_lines(), 'utf-8')
  reader = csv.DictReader(text, delimiter=',', quotechar='"')
  for row in reader:
    products.append(row)

  categories = sorted(set([product['category'] for product in products]))
  categories = [{"category_id": i+1, "name": category} for (i, category) in enumerate(categories)]
  categories_by_name = {category["name"]: category["category_id"] for category in categories}
  products = [{"product_id": i+1, 
              "name": product["name"], 
              "price": float(product["price"]), 
              "category_id": categories_by_name[product["category"]],
              "provider_id": random.choice(providers_info)["provider_id"]} 
              for (i, product) in enumerate(products)]
  return {
      'products': products,
      'categories': categories
  }

def build_people_dataset(number):

  people = []
  addresses = []
  payment_info = []
  address_id = 0
  payment_id = 0

  for i in range(1, number+1):
    # Person data
    people.append({
      "person_id": i,
      "first_name": fake.first_name(),
      "last_name": fake.last_name(),
      "birth_date": fake.date_between_dates(datetime(1960, 1, 1), datetime(2002, 6, 1)),
      "email": fake.email(),
      "phone": fake.phone_number(),
      "username": fake.user_name(),
      "password": fake.sha256(),
      "job": fake.job()
    })

    # Payment information
    if random.choice([False]*1 + [True]*2):
      payment_id += 1
      payment_info.append({
          "payment_id": payment_id,
          "person_id": i,
          "expiration": fake.credit_card_expire(),
          "number": fake.credit_card_number(),
          "provider": fake.credit_card_provider(),
          "security_code": fake.credit_card_security_code()
      })

    # Registered addresses
    for j in range(random.choice([1]*43 + [2]*6 + [3])):
      address_id+=1
      addresses.append(
      {
        "address_id": address_id,
        "person_id": i,
        "city": fake.city(),
        "number": fake.building_number(),
        "country": "España",
        "zipcode": fake.postcode(),
        "street": fake.street_name()
      })

  return {
      "people": people,
      "addresses": addresses,
      "payment_information": payment_info,
  }

def build_network_dataset(people_info):

  WEB_PAGES = [fake.uri_path() for i in range(0,100)]
  ACCESS_METHOD_PROPORTION = ['GET'] * 10 + ['POST'] 
  pages = []
  accesses = []
  access_id = 0

  for i in range(0, len(WEB_PAGES)):
    pages.append({
        "page_id": i+1,
        "path": WEB_PAGES[i]
    })

  for person in people_info:
    # Access to webpages
    for j in range(int(random.gauss(60, 40))):
      access_id += 1
      accesses.append({
          "access_id": access_id,
          "person_id": person["person_id"],
          "method": random.choice(ACCESS_METHOD_PROPORTION),
          "ip": fake.ipv4_public(),
          "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
          "page_id": random.randint(1, len(WEB_PAGES)-1)
      })

  # Anonymous access
  for i in range(int(random.gauss(1000, 100))):
    access_id += 1
    accesses.append({
        "access_id": access_id,
        "person_id": None,
        "method": random.choice(ACCESS_METHOD_PROPORTION),
        "ip": fake.ipv4_public(),
        "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
        "page_id": random.randint(1, len(WEB_PAGES)-1)
    })

  return {
    "web_pages":  pages,
    "accesses": accesses
  }

def build_shopping_dataset(people, products, people_addresses):

  shopping_carts = []
  shopping_cart_products = []
  orders = []
  order_products = []
  invoices = []
  cart_id = 0
  shopping_cart_id = 0
  order_id = 0
  order_product_id = 0
  invoice_id = 0

  PRODUCTS_PROBABILITY = [1]*2 + [2] * 3 + [3] * 3 + [4]*2 + [5]
  ORDER_PROBABILITY = [0]+[1]*7+[2]*3+[3]*3+[4]*2+[5]
  QUANTITY_PROBABILITY = [1]*5 +[2]*2 +[3]
  RATING_PROBABILITY = [1]+[2]+[3]*2+[4]*4+[5]*3

  for person in people:
    # Build shopping cart
    if random.choice([False * 9] + [True]):
      cart_id += 1
      shopping_carts.append({
          "cart_id": cart_id,
          "person_id": person["person_id"],
          "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
      })

      chosen = random.sample(products, k = random.choice(PRODUCTS_PROBABILITY))
      for product in chosen:
        shopping_cart_id += 1
        shopping_cart_products.append({
            "cart_id": cart_id,
            "product_id": product["product_id"],
            "quantity": random.choice(QUANTITY_PROBABILITY)
        })
    
    # Build orders
    for i in range(0, random.choice(ORDER_PROBABILITY)):
      order_id += 1
      order_price = 0
      chosen = random.sample(products, k = random.choice(PRODUCTS_PROBABILITY))
      for product in chosen:
        order_product_id += 1
        quantity = random.choice(QUANTITY_PROBABILITY)
        order_products.append({
            "order_id": order_id,
            "product_id": product["product_id"],
            "quantity": quantity
        })
        order_price += quantity * product['price']

      person_addresses = [address for address in people_addresses if address["person_id"] == person["person_id"]]
      delivery_address = random.choice(person_addresses)
      billing_address = random.choice(person_addresses)
      orders.append({
          "order_id": order_id,
          "person_id": person["person_id"],
          "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
          # Purposely left wrong
          "delivery_address": delivery_address['address_id'],
          "billing_address": billing_address['address_id'],
          "price": order_price
      })

  # Build invoices
  for order in random.choices(orders, k = int(len(orders) * 0.8)):
    invoice_id += 1
    invoices.append({
      "invoice_id": invoice_id,
      "order_id": order["order_id"],
      "date": fake.date_time_between(order["date"], datetime(2020,9,1,23,59,59)),
      "rating": random.choice(RATING_PROBABILITY)
    })

  return {
      'carts': shopping_carts,
      'cart_product': shopping_cart_products,
      'orders': orders,
      'order_product': order_products,
      'invoices': invoices    
  }

dataset = {}
dataset.update(build_providers_dataset(NUMERO_PROVEEDORES))
dataset.update(build_products_dataset(dataset['providers']))
dataset.update(build_people_dataset(NUMERO_CLIENTES))
dataset.update(build_network_dataset(dataset['people']))
dataset.update(build_shopping_dataset(dataset['people'], dataset['products'], dataset['addresses']))

con = pymysql.connect(host='localhost', user='admin',password='Password0', database='shop')
try:
    for table in dataset:
      first_time = True
      sql = ""
      with con.cursor() as cur:        
        for entity in dataset[table]:
          if (first_time):
            first_time = False
            str_columns = ",".join(entity.keys())
            str_values = ",".join(["%s"] * len(entity.keys()))
            sql = f"INSERT INTO {table} ({str_columns}) VALUES ({str_values})"
          cur.execute(sql, tuple(entity.values()))
        con.commit()
finally:
    con.close()

USER = "admin"
PASSWORD = "Password0"
HOST = "localhost"
DATABASE = "shop"
conn = create_engine(f'mysql+pymysql://{USER}:{PASSWORD}@{HOST}/{DATABASE}')
pd.read_sql_query("SELECT * FROM products LIMIT 3", conn)

IntegrityError: (1062, "Duplicate entry '1' for key 'providers.PRIMARY'")

Parece que el código pisa la base de datos creada porque no se está corriendo el script asociado, hacer que python corra el script cuando toca

La base de datos shop tiene una tabla people con información sobre los clientes de la tienda ficticia. Escribe el código Python necesario para inicializar un DataFrame con el contenido de la tabla:

## EJERCICIO 2

La siguiente URL https://datosabiertos.carm.es/odata/Agricultura/IMIDA_dia_2018.csv contiene el informe meteorológico diario de las diferentes estaciones meteorológicas de la Región de Murcia a lo largo del año 2018. Observa el contenido del fichero csv y a continuación utiliza la función read_csv de pandas sobre esta URL con los  parámetros necesarios (header, sep, decimal, quotechar y encoding).

## EJERCICIO 3

El Instituto de Salud Carlos III ofrece gratuitamente información actualizada sobre la situación del COVID-19 en España. En la siguiente URL podemos obtener un fichero csv actualizado con los casos positivos notificados por las Comunidades Autónomas a nivel provincial: https://cnecovid.isciii.es/covid19/resources/casos_diagnostico_provincia.csv. En el siguiente enlace se nos describe este conjunto de datos y se nos proporcionan otros conjuntos de datos de interés.

Utiliza el método read_csv con los parámetros adecuados para obtener el DataFrame con los casos positivos por provincia. Muestra a continuación su contenido:

## EJERCICIO 4

La siguiente URL contiene información de películas estadounidenses obtenidas de la Wikipedia en formato JSON: https://raw.githubusercontent.com/prust/wikipedia-movie-data/master/movies.json. Utiliza el método `read_json` de `pandas` para cargar su contenido en un DataFrame:

## EJERCICIO 5

Haciendo uso de la librería `requests` y `BeautifulSoup`, accede a la siguiente URL https://catalogoreina.com/859-grifos-cocina-roca y recupera el nombre de los artículos mostrados:

## EJERCICIO 6

Haciendo uso de la librería `requests` y `BeautifulSoup`, accede a la página web del DB-Engines https://db-engines.com/en/ranking y recupera los nombres de las 10 bases de datos más populares:

PISTA: el selector CSS [`nth-child`](https://developer.mozilla.org/es/docs/Web/CSS/:nth-child) puede serte de utilidad.

OTRA PISTA: para evitar que salga el contenido de la etiqueta `info` junto al nombre de la base de datos, haz uso del método [`extract`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract).

## EJERCICIO 7

Haciendo uso de la librería `requests` y `BeautifulSoup`, recupera el nombre, los acabados, las medidas y el plazo de entrega del siguiente artículo: https://catalogoreina.com/nuestras-marcas-muebles-bano/10233-mueble-bano-con-patas-althea-moderno-3-cajones.html

## <ins>Ejercicios Opcionales </ins>

## EJERCICIO 8

Esta base de datos también contiene una tabla orders con información de la cabecera de los pedidos de la tienda. En esta tabla existe una columna price que almacena el importe total del pedido. Construye a continuación un DataFrame con las cabeceras de pedidos ordenada por importe de manera descendente:

## EJERCICIO 9

La base de datos también contiene las tablas web_pages y accesses con información de las páginas web de la empresa y de los accesos realizados a ellas respectivamente. Ambas tablas pueden relacionarse por el campo page_id. Inicializa un DataFrame que contenga los campos page_id y path de web_pages y el total de accesos realizado sobre cada una de ellas:

## EJERCICIO 10

Crea un diccionario Python que represente una factura en formato JSON:

* Contendrá 4 campos cuyos valores puedes inventarte, pero siguiendo las siguientes indicaciones:
  * **client_id**: identificador de cliente, de tipo cadena.
  * **products**: array con al menos dos productos de tipo objeto. Cada objeto tendrá dos campos:
    * **name**: de tipo cadena.
    * **price**: de tipo numérico.
  * **date**: de tipo cadena.
  * **address**: de tipo objeto, con los siguientes campos:
    * **street**: de tipo objeto, con los siguientes campos:
      * **name**: de tipo cadena.
      * **number**: de tipo numérico.
    * **zipcode**: de tipo numérico.

El diccionario tendrá que tener un formato tal que sea aceptado por el siguiente [validador](https://jsonlint.com/) de contenido JSON.

## EJERCICIO 11

Utilizando la librería PyPDF4, recupera el número de páginas del siguiente PDF:
```
PDF_URL = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_278_COVID-19.pdf'
```

## EJERCICIO 12

Utilizando la librería PyPDF4, recupera la fecha de creación del documento (campo /CreationDate de los metadatos):

## EJERCICIO 13

Extrae la información del pdf en tablas y obten aquella que contiene los Detalles de los quince países con más casos confirmados fuera de Europa.

PISTA: esta tabla es la última del documento PDF. Utiliza la función len para obtener el total de tablas extraídas y saber cuál de ellas seleccionar.

## EJERCICIO 14

Mediante `read_excel`, carga el contenido de todas las hojas del fichero excel "Orders-With Nulls.xlsx", muestra el nombre de todas las hojas y muestra la hoja denominada `Summary`: